In [1]:

import sys
print(sys.executable)

/home/s.wendelken/.conda/envs/ehr-env/bin/python


In [7]:
from huggingface_hub import login
from config import HF_TOKEN
login(token=HF_TOKEN)

!huggingface-cli download meta-llama/Meta-Llama-3.1-8B-Instruct --include "original/*" --local-dir /work/AI-EHR/discharge_me/models/Meta-Llama-3-1-8B-Instruct

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/s.wendelken/.cache/huggingface/token
Login successful
Fetching 3 files:   0%|                                   | 0/3 [00:00<?, ?it/s]Downloading 'original/tokenizer.model' to '/work/AI-EHR/discharge_me/models/Meta-Llama-3-1-8B-Instruct/.huggingface/download/original/tokenizer.model.82e9d31979e92ab929cd544440f129d9ecd797b69e327f80f17e1c50d5551b55.incomplete'

original/params.json: 100%|█████████████████████| 199/199 [00:00<00:00, 766kB/s]
Download complete. Moving file to /work/AI-EHR/discharge_me/models/Meta-Llama-3-1-8B-Instruct/original/params.json

tokenizer.model: 100%|█████████████████████| 2.18M/2.18M [00:00<00:00, 81.2MB/s]

consolidated.00.pth:   0%|                          | 0.00/16

In [8]:
!huggingface-cli download meta-llama/Meta-Llama-3.1-70B-Instruct --include "original/*" --local-dir /work/AI-EHR/discharge_me/models/Meta-Llama-3-1-70B-Instruct

Fetching 10 files:   0%|                                 | 0/10 [00:00<?, ?it/s]Downloading 'original/consolidated.04.pth' to '/work/AI-EHR/discharge_me/models/Meta-Llama-3-1-70B-Instruct/.huggingface/download/original/consolidated.04.pth.28953d0f763ddb42f21b01f15f7d3a33f4eac011394a1bff802def1b354d08f7.incomplete'

tokenizer.model:   0%|                              | 0.00/2.18M [00:00<?, ?B/s]

consolidated.04.pth:   0%|                          | 0.00/17.6G [00:00<?, ?B/s]Downloading 'original/consolidated.00.pth' to '/work/AI-EHR/discharge_me/models/Meta-Llama-3-1-70B-Instruct/.huggingface/download/original/consolidated.00.pth.4582a6ecf14858cfa7899a367fd0194449e2967d6966c25ac231295b69845426.incomplete'



original/params.json: 100%|█████████████████████| 199/199 [00:00<00:00, 894kB/s]
Download complete. Moving file to /work/AI-EHR/discharge_me/models/Meta-Llama-3-1-70B-Instruct/original/params.json
tokenizer.model: 100%|█████████████████████| 2.18M/2.18M [00:00<00:00, 42.7MB/s]
Down

In [9]:
!huggingface-cli download meta-llama/Meta-Llama-3.1-405B-Instruct-FP8 --include "original/*" --local-dir /work/AI-EHR/discharge_me/models/meta-llama/Meta-Llama-3-1-405B-Instruct-FP8
# meta-llama/Meta-Llama-3.1-405B-Instruct-FP8

Fetching 123 files:   0%|                               | 0/123 [00:00<?, ?it/s]Downloading 'original/consolidated.00/consolidated-00008-of-00013.pth' to '/work/AI-EHR/discharge_me/models/meta-llama/Meta-Llama-3-1-405B-Instruct-FP8/.huggingface/download/original/consolidated.00/consolidated-00008-of-00013.pth.44a0df4a6c779a633742578f200842186830f7a430863815e20442c55a52b234.incomplete'

consolidated-00001-of-00013.pth:   0%|              | 0.00/4.97G [00:00<?, ?B/s]Downloading 'original/consolidated.00/consolidated-00006-of-00013.pth' to '/work/AI-EHR/discharge_me/models/meta-llama/Meta-Llama-3-1-405B-Instruct-FP8/.huggingface/download/original/consolidated.00/consolidated-00006-of-00013.pth.24886ea24897c7a18416fc4773f9ca13fc6fb2dc04023c54df28b91bedc75dcf.incomplete'


consolidated-00002-of-00013.pth:   0%|              | 0.00/4.95G [00:00<?, ?B/s]


original/README.md: 100%|██████████████████| 1.52k/1.52k [00:00<00:00, 5.25MB/s]
Download complete. Moving file to /work/AI-EHR/discharge_

In [3]:
import transformers
import torch

model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)

messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"},
]

outputs = pipeline(
    messages,
    max_new_tokens=256,
)
print(outputs[0]["generated_text"][-1])


ImportError: Using `low_cpu_mem_usage=True` or a `device_map` requires Accelerate: `pip install accelerate`

In [4]:
# This example showcases how to leverage the 3.1 8B Instruct models using 
# torch.compile to accelerate inference.
#
# You need CUDA and torch >= 2.3 in order to run this example.

import os
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
os.environ["TOKENIZERS_PARALLELISM"] = "false" # silence warnings when compiling

device = "cuda"
ckpt = "meta-llama/Meta-Llama-3.1-8B-Instruct"

model = AutoModelForCausalLM.from_pretrained(ckpt, torch_dtype=torch.float16)
model.to(device)

tokenizer = AutoTokenizer.from_pretrained(ckpt)

prompt = "Why dogs are so cute?"
inputs = tokenizer(prompt, return_tensors="pt").to(device)

# Specify the max length (including both the prompt and the response)
# When calling `generate` with `cache_implementation="static" later, this is also used to create a `StaticCache` object
# with sequence length = `max_length`. The longer the more you will re-use it
model.generation_config.max_length = 128

# without `torch.compile`: each call takes ~ 5.0 seconds (on A100 80G + torch 2.3)
outputs = model.generate(**inputs, do_sample=False)
response = tokenizer.batch_decode(outputs)[0]
print(response)

# `torch.compile(model, ...)` is not recommended as you compile callbacks
# and full generate. We recommend compiling only the forward for now. 
# "reduce-overhead" will use cudagraphs. 
model.forward = torch.compile(model.forward, mode="reduce-overhead", fullgraph=True)
model.generation_config.cache_implementation = "static"

# with `torch.compile` (on A100 80G + torch 2.3)
# 1st call: ~ 90 seconds
outputs = model.generate(**inputs, do_sample=False)
response = tokenizer.batch_decode(outputs)[0]
# 2nd call: ~ 60 seconds
outputs = model.generate(**inputs, do_sample=False)
response = tokenizer.batch_decode(outputs)[0]
# 3nd call: ~ 1.5 seconds
outputs = model.generate(**inputs, do_sample=False)
response = tokenizer.batch_decode(outputs)[0]
print(response)

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [2]:
!pip install --upgrade transformers
!pip install accelerate`

/bin/bash: -c: line 1: unexpected EOF while looking for matching ``'
/bin/bash: -c: line 2: syntax error: unexpected end of file
